## Pandas exercise
______________________

For those of you who are interested in challenging yourselves, here's a quick exercise in manipulating data matrices into a workable format.

By default, [kallisto](https://pachterlab.github.io/kallisto/) outputs a counts matrix for each sample. This means that for each sample that you've processed you generated a separate file. What do these individual files look like? 

In [2]:
import pandas as pd

In [8]:
f = 'kallisto_counts/Control_78_Day_0_Rep3.tsv'
df = pd.read_csv(f, sep='\t') 
df.head()

,target_id,length,eff_length,est_counts,tpm
0,ENST00000456328.2,1657,1548.4200,0.078546,0.004365
1,ENST00000450305.2,632,523.4170,0.000000,0.000000
2,ENST00000488147.1,1351,1242.4200,5.916130,0.409730
3,ENST00000619216.1,68,13.5858,0.000000,0.000000
4,ENST00000469289.1,535,426.5120,0.000000,0.000000


The data in each of these files have information about expression on the **transcript level**. However we want to examine expression of **genes**, not transcripts. 

Additionally, the expression profiles of each sample are separated out by sample. We want to concatenate our matrices such that we have a **gene by sample** expression matrix. We can examine the counts matrix that was provided for us to get a rough idea of what our goal is.

In [9]:
f = 'kallisto_counts/fshd.counts.matrix.csv'
df = pd.read_csv(f, sep=',')
df.head()

,Unnamed: 0,Control_78_Day_0_Rep3,Control_78_Day_0_Rep4,Control_78_Day_3_Rep1,Control_78_Day_3_Rep2,FSHD2_19_Day_0_Rep1,FSHD2_19_Day_0_Rep2,FSHD2_19_Day_3_Rep1,FSHD2_19_Day_3_Rep2
0,ENSG00000223972.5|DDX11L1,0,2,0,0,0,0,0,0
1,ENSG00000227232.5|WASH7P,6,20,6,13,9,6,0,14
2,ENSG00000278267.1|MIR6859-1,0,0,0,0,0,0,0,0
3,ENSG00000243485.5|MIR1302-2HG,0,0,0,0,0,0,0,0
4,ENSG00000284332.1|MIR1302-2,0,0,0,0,0,0,0,0


Finally I will provide one more resource. We need a way to map transcript IDs to gene IDs. This file is provided at `/pub/erebboah/cosmos/FSHD_bulkRNA/ref/hg38.gene_transcripts.tab`. This is what the file looks like. Here, 'tid' stands for transcript ID, 'gname' stands for gene name, and 'gid' stands for gene ID.

In [11]:
f = 'ref/hg38.gene_transcripts.tab'
df = pd.read_csv(f, sep='\t', header=None, 
                 usecols=[0,1,2], 
                 names=['gid', 'gname', 'tid'])
df.head()

,gid,gname,tid
0,ENSG00000223972.5,DDX11L1,ENST00000456328.2
1,ENSG00000223972.5,DDX11L1,ENST00000450305.2
2,ENSG00000227232.5,WASH7P,ENST00000488147.1
3,ENSG00000278267.1,MIR6859-1,ENST00000619216.1
4,ENSG00000243485.5,MIR1302-2HG,ENST00000469289.1


A few closing notes:
* use the `est_counts` column from the kallisto output files 
* you can either use gene ID or gene name (or both!) as your gene identifier, gene IDs are typically more reliable though because there are some sneaky duplicate gene IDs for different genes 💀

Best of luck!